<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformer_Models_for_Time_Series_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Transformer model for time series forecasting
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_size, num_heads, num_layers, d_model, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.input_embedding = nn.Linear(input_size, d_model)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(
            self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, src):
        src = self.input_embedding(src)
        output = self.transformer_encoder(src)
        return self.fc(output[-1])

# Generate synthetic time series data
def generate_synthetic_data(n_samples, seq_len):
    x = np.linspace(0, 4 * np.pi, seq_len)
    y = np.sin(x) + 0.1 * np.random.randn(n_samples, seq_len)
    return y

# Prepare the data
n_samples = 1000
seq_len = 30
data = generate_synthetic_data(n_samples, seq_len)
X = torch.tensor(data[:, :-1], dtype=torch.float32).unsqueeze(2)
y = torch.tensor(data[:, -1], dtype=torch.float32).unsqueeze(1)  # Ensure y has shape (1000, 1)

# Model instantiation and training
model = TimeSeriesTransformer(input_size=1, num_heads=4, num_layers=2, d_model=64)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    output = model(X.permute(1, 0, 2))
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

print(f"Training loss: {loss.item()}")